# Implementation of passMan

### Import, declaring hash, charmap, and clipboard functions

In [1]:
import json
import os
import random
import pyperclip
import hashlib
SRandInt = random.SystemRandom().randint
def getTextFromClipboard():
    return pyperclip.paste()

def pasteTextToClipboard(text):
    pyperclip.copy(text)
    return True

def dec_to_base(num,base=None, special_chars=False):  #Maximum base - 36
    new_num    = ""
    char_set_2  =['!@#$%^&*()_-;:<>?/\|', 'abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', '0123456789']
    char_set    = char_set_2[0:] if special_chars else char_set_2[1:]
    chrs        =''.join(char_set)
    if base is None:
        base    = len(chrs)
    while num>0:
        dig     = int(num%base)
        new_num = chrs[dig % len(chrs)] + new_num
        num     //= base
    new_num = new_num[::-1]  #To reverse the string
    return new_num

def getHash(string):
    return int(int(hashlib.sha256(string.encode('utf-8')).hexdigest(), 16)**2)

def getSeed(n=1000):
    return ''.join([str(SRandInt(0, 9)) for I in range(n)])

### functions
- passFile is a dictionary of type `{seed: seed, passwords : { '1' : {'website':...`
- passFile_toList is a function that converts passFile to a list, and allows selection using website name

In [2]:
def add_passDict_to_passFile(passDict, passFile):
    index = max([int(k) for k in passFile['passwords'].keys()]) + 1 if len(passFile['passwords'].keys()) > 0 else 1
    passFile['passwords'][index] = passDict
    return passFile

def get_passFile(fileName='pass.json'):
    passFile = None
    seed=getSeed()
    if not os.path.isfile(fileName):
        with open(fileName, 'w') as f:
            json.dump({'seed': seed, 'passwords': {}}, f)
    with open(fileName, 'r') as f:
        passFile = json.load(f)
    return passFile

def DeletePassFile(fileName='pass.json'):
    if os.path.isfile(fileName):
        os.remove(fileName)
        return True
    else:
        return False

def add_to_passFile(passDicts=[], fileName='pass.json'):
    passFile        = get_passFile(fileName)    
    for passDict in passDicts:
        passFile = add_passDict_to_passFile(passDict, passFile)

    with open(fileName, 'w') as f:
        json.dump(passFile, f)
    return passFile

def create_pwd_dict(username, website, special_chars=False, length=14):
    return {'username': username, 'website': website, 'length': length, 'special_chars': special_chars}

def get_password(index, passFile, website=None, username=None):
    if type(index) is int:
        index = str(index)
    username, website, seed = passFile['passwords'][index]['username'], passFile['passwords'][index]['website'], passFile['seed']
    hStr = username + website + str(index) + str(getHash(seed)) + seed + seed[::-1]
    return dec_to_base(getHash(hStr), special_chars=passFile['passwords'][index]['special_chars'])[:passFile['passwords'][index]['length']]

def passFile_to_list(passFile, website=None):    
    if website is None:
        return [[passFile['passwords'][k]['website'] , passFile['passwords'][k]['username'],get_password(k, passFile)] for k in passFile['passwords'].keys()]
    l = passFile_to_list(passFile)
    return [i for i in l if website in i[0]]

def get_password_for_website(website, passFile=None):
    if passFile is None:
        passFile = get_passFile()
    return passFile_to_list(passFile=passFile, website=website)


### Adding top level functions

In [3]:
def AddPassword(username, website, length=14, special_chars=False, passFile=None):
    add_to_passFile(passDicts=[create_pwd_dict(username=username, website=website, length=length, special_chars=special_chars)] , fileName='pass.json')

def GetPasswords(website=None, username=None, passFile=None):
    if website is None:
        website = input('Website: ')
        if website.strip() == '':
            website = getTextFromClipboard()
    if passFile is None:
        passFile = get_passFile()
    pwd = get_password_for_website(website, passFile)
    if pwd is None:
        print('No passwords found for ' + website)
        # return -1
    elif len(pwd) == 1:
        print('Copied to clipboard!')
        pasteTextToClipboard(pwd[0][2])
        # return 0
    else:
        print('Multiple passwords found for ' + website)
        print('Specify Username:')
        if username is None:
            username = input('Username: ')
            if username.strip() == '':
                username = getTextFromClipboard()
        l = [i for i in pwd if i[1] == username]
        if len(l) == 1:
            print('Copied to clipboard!')
            pasteTextToClipboard(l[0][2])
        elif len(l) == 0:
            print('No passwords found for ' + website + ' with username ' + username)
        else:
            print('Multiple passwords found for ' + website + ' and username ' + username)
            print('Copying most recent password')
            pasteTextToClipboard(l[-1][2])

## Breakdown and runnning the notebook

In [4]:
getHash('hello') # get integer value of the sha256 hash of the string 'hello'

413303971471188822875652362255993467374868283919722305284465395822324048964002785045033324791520964001278634827618252814355326521394268033432013378864400

In [5]:
dec_to_base(getHash('hello'), special_chars=False) 

'CkOpAWP9gYlDIQnFyplt6ZchKD7bV3xSigCwNKsSOcXTLbgTyTsZGLhQR2XiCvEOTXrHvJHqcIIfQl8IKPbLZb'

In [6]:
dec_to_base(getHash('hello'), special_chars=True) 

'\\76Dl#SdOD9/-I_uLT-OTYq#B/iJfa?5e5vv^_Ahn@?yR>\\Z;g4)wJM!GBa5D#<cc;$%a7_-O&t>6IDg'

In [7]:
create_pwd_dict(username='userA', website='websiteA', length=14)

{'username': 'userA',
 'website': 'websiteA',
 'length': 14,
 'special_chars': False}

In [8]:
# Demonstates using AddPassword to add different types of passwords
AddPassword(username='user1', website='github.com', special_chars=True)
AddPassword(username='user2', website='google.com', length=30)
AddPassword(username='user3', website='facebook.com')

passFile_to_list(get_passFile())

[['github.com', 'user1', 'xJKx$0^9amLXM('],
 ['google.com', 'user2', 'kHRMUGPgwzpeMLhKrfme73KKR0aEGk'],
 ['facebook.com', 'user3', 'GYM4fVKFElhKiv']]

In [9]:
GetPasswords(website='google.com') # copies result to clipboard

Copied to clipboard!


In [10]:
get_password_for_website(website='facebook.com') # not secure, only used for testing and internal functions

[['facebook.com', 'user3', 'GYM4fVKFElhKiv']]

In [11]:
# finally delete the passFile if you're testing. if not, ignore this bit
DeletePassFile('pass.json')